In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
from cv2 import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense,MaxPooling2D,Conv2D,Flatten,Dropout,BatchNormalization,Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns

In [2]:
fldr="./UTKFace"
files=os.listdir(fldr)

ages=[]
genders=[]
images=[]

for fle in files:
    age=int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    
    total=fldr+'/'+fle
    image=cv2.imread(total)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(48,48))
    
    images.append(np.array(image))
    ages.append(np.array(age))
    genders.append(gender)
    
ages = np.array(ages)
genders = np.array(genders)
images=np.array(images)
age_groups=np.where((ages>=0)&(ages<13),0,
                    np.where((ages>=13)&(ages<20),1,
                             np.where((ages>=20)&(ages<60),2,3)))

np.save('/kaggle/working/image.npy',images)
np.save('/kaggle/working/age.npy',ages)
np.save('/kaggle/working/gender.npy',genders)
np.save('/kaggle/working/age_group.npy',age_groups)

In [3]:
labels=[]
i=0
while i<len(ages):
    label=[]
    label.append(ages[i])
    label.append(genders[i])
    label.append(age_groups[i])
    labels.append(label)
    i=i+1

images_f=images/255
labels=np.array(labels)

In [4]:
X_train,X_val,Y_train,Y_val=train_test_split(images_f,labels,test_size=0.20)


In [7]:
Y_train_2=[Y_train[:,0],Y_train[:,1],Y_train[:,2]]
Y_val_2=[Y_val[:,0],Y_val[:,1],Y_val[:,2]]

In [ ]:
def Convolution(input_tensor,filters):
    x=Conv2D(filters=filters,kernel_size=(3,3),padding="same",strides=(1,1),kernel_regularizer=l2(0.01))(input_tensor)
    x=BatchNormalization()(x)
    x=Dropout(0.1)(x)
    x=Activation('relu')(x)
    return x

def model(input_shape):
    inputs=Input((input_shape))
    conv_1=Convolution(inputs,32)
    maxp_1=MaxPooling2D(pool_size=(2,2))(conv_1)
    conv_2=Convolution(maxp_1,64)
    maxp_2=MaxPooling2D(pool_size=(2,2))(conv_2)
    conv_3=Convolution(maxp_2,128)
    maxp_3=MaxPooling2D(pool_size=(2,2))(conv_3)
    conv_4=Convolution(maxp_3,256)
    maxp_4=MaxPooling2D(pool_size=(2,2))(conv_4)
    conv_5=Convolution(maxp_4,512)
    maxp_5=MaxPooling2D(pool_size=(2,2))(conv_5)
    flatten=Flatten()(maxp_5)
    dense_1=Dense(128,activation='relu')(flatten)
    dense_2=Dense(128,activation='relu')(flatten)
    dense_3=Dense(128,activation='relu')(flatten)
    drop_1=Dropout(0.3)(dense_1)
    drop_2=Dropout(0.2)(dense_2)
    drop_3=Dropout(0.3)(dense_3)
    output_1=Dense(1,activation='linear',name='age_out')(drop_1)
    output_2=Dense(1,activation='sigmoid',name='gender_out')(drop_2)
    output_3=Dense(4,activation='softmax',name='category_out')(drop_3)
    model=Model(inputs=[inputs],outputs=[output_1,output_2,output_3])
    model.compile(loss=["mse","binary_crossentropy","SparseCategoricalCrossentropy"],optimizer=Adam(learning_rate=0.001,clipnorm=1.0),metrics={'age_out':"mae",'gender_out':"accuracy",'category_out':"SparseCategoricalAccuracy"})
    return model

Model=model((48,48,3))
Model.summary()

In [ ]:
fle_s='Age_Group_Detection.h5'
checkpoint=ModelCheckpoint(fle_s,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=25,monitor='val_loss',restore_best_weights='True')
callback_list=[checkpoint,Early_stop]
History=Model.fit(X_train,Y_train_2,batch_size=64,validation_data=(X_val,Y_val_2),epochs=250,callbacks=callback_list)

In [ ]:
Model.evaluate(X_val,Y_val_2)

In [ ]:
pred=Model.predict(X_val)

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.xlabel=('Epoch')
plt.ylabel=('Loss')
plt.legend(['Train','Validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

In [ ]:
plt.plot(History.history['age_out_mae'])
plt.plot(History.history['val_age_out_mae'])
plt.title('Model Accuracy')
plt.xlabel=('Epoch')
plt.ylabel=('Loss')
plt.legend(['Train','Validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

In [ ]:
plt.plot(History.history['gender_out_accuracy'])
plt.plot(History.history['val_gender_out_accuracy'])
plt.title('Model Accuracy')
plt.xlabel=('Epoch')
plt.ylabel=('Loss')
plt.legend(['Train','Validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

In [ ]:
fig,ax=plt.subplots()
ax.scatter(Y_val_2[0],pred[0])
ax.plot([Y_val_2[0].min(),Y_val_2[0].max()],[Y_val_2[0].min(),Y_val_2[0].max()],'k--',lw=4)
ax.set_xlabel('Actual Age')
ax.set_ylabel('Predicted Age')
plt.show()

In [ ]:
i=0
Pred_l=[]
while(i<len(pred[2])):
    Pred_l.append(int(np.argmax(pred[2][i])))
    i=i+1

In [ ]:
report=classification_report(Y_val_2[2],Pred_l)
print(report)

In [ ]:
results=confusion_matrix(Y_val_2[2],Pred_l)
sns.heatmap(results,annot=True,fmt='.0f')

In [ ]:
def test_image(ind,images,images_f,Model):
    plt.imshow(images[ind])
    image_test=images_f[ind]
    pred_l=Model.predict(np.array([image_test]))
    gender_f=['Male','Female']
    age=int(np.round(pred_l[0][0]))
    gender=int(np.round(pred_l[1][0]))
    category_f=['Child','Teenage','Adult','Aged']
    category=int(np.argmax(pred_l[2]))
    print("Predicted Age is "+str(age)+".")
    print("Predicted Gender is "+gender_f[gender]+".")
    print("Predicted Category is "+category_f[category]+".")

In [ ]:
test_image(4,images,images_f,Model)

In [ ]:
test_image(457,images,images_f,Model)

In [ ]:
test_image(3621,images,images_f,Model)

In [ ]:
test_image(583,images,images_f,Model)

In [ ]:
test_image(225,images,images_f,Model)

In [ ]:

test_image(9,images,images_f,Model)

In [ ]:
test_image(167,images,images_f,Model)

In [ ]:
test_image(26,images,images_f,Model)